In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Import packages
import pandas as pd
import json
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense, GRU
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorboard.plugins.hparams import api as hp
import numpy as np

In [3]:
# Convert JSON file to array of JSON objects
with open('./data/Sarcasm_Headlines_Dataset.json', 'r') as f:
    data = json.loads("[" + f.read().replace("}\n{", "},\n{") + "]")

In [4]:
# Create DataFrame from array of JSON objects
df = pd.DataFrame(data)

In [5]:
# Create predictor and target from DataFrame
X = df['headline']
y = df['is_sarcastic'].values.reshape(-1, 1)

In [6]:
# Create train/test split
X_train_all, X_test, y_train_all, y_test = train_test_split(X, y, test_size=0.2)

In [7]:
# Set tokenizer and padder parameters
num_words = 1000
oov_token = '<OOV>'
pad_type = 'pre'
trunc_type = 'post'
embedding_dim = 8

In [8]:
# Fit tokenizer on the training set
tokenizer = Tokenizer(num_words=num_words, oov_token=oov_token)
tokenizer.fit_on_texts(X_train_all)
word_index = tokenizer.word_index

In [9]:
# Tokenize the training and test sets
X_train_sequences_all = tokenizer.texts_to_sequences(X_train_all)
X_test_sequences = tokenizer.texts_to_sequences(X_test)

maxlen = max([len(x) for x in X_train_sequences_all])

In [10]:
# Pad the training and test sets
X_train_padded_all = pad_sequences(X_train_sequences_all, padding=pad_type, truncating=trunc_type, maxlen=maxlen)
X_test_padded = pad_sequences(X_test_sequences, padding=pad_type, truncating=trunc_type, maxlen=maxlen)

In [11]:
# Create a validation set
X_train_padded, X_val_padded, y_train, y_val = train_test_split(X_train_padded_all, y_train_all, test_size=0.25)

## Hyperparameter tuning

In [12]:
! rm -rf ./logs/

In [13]:
HP_EMBED_INITIALIZER = hp.HParam('embeddings_initializer', hp.Discrete(["Constant",
"GlorotNormal",
"GlorotUniform",
"HeNormal",
"HeUniform",
"Identity",
"Initializer",
"LecunNormal",
"LecunUniform",
"Ones",
"Orthogonal",
"RandomNorma",
"RandomUniform",
"TruncatedNormal",
"VarianceScaling",
"Zeros",
"constant",
"glorot_normal",
"glorot_uniform",
"he_normal",
"he_uniform",
"identity",
"lecun_normal",
"lecun_uniform",
"ones",
"orthogonal",
"random_normal",
"random_uniform",
"truncated_normal",
"variance_scaling",
"zeros"]))

HP_EMBEDDINGS_REGULARIZER = hp.HParam('l2 regularizer', hp.RealInterval(0.0, 1.0))

HP_EMBEDDINGS_CONSTRAINT = hp.HParam('embeddings_constraint', hp.Discrete(["MaxNorm",
"MinMaxNorm",
"NonNeg",
"RadialConstraint",
"UnitNorm",
"max_norm",
"min_max_norm",
"non_neg",
"radial_constraint",
"unit_norm"]))

HP_ACTIVATION_D1 = hp.HParam('activation_d1', hp.Discrete([
    "relu",
    "sigmoid",
    "softmax",
    "softplus",
    "softsign",
    "tanh",
    "selu",
    "elu",
    "exponential"]))

HP_ACTIVATION_D2 = hp.HParam('activation_d2', hp.Discrete([
    "relu",
    "sigmoid",
    "softmax",
    "softplus",
    "softsign",
    "tanh",
    "selu",
    "elu",
    "exponential"]))

HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(["sgd",
    "rmsprop",
    "adam",
    "adadelta",
    "adagrad",
    "adamax",
    "nadam",
    "ftrl"]))

METRIC_ACCURACY = 'val_accuracy'

with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
    hp.hparams_config(
        hparams=[HP_EMBED_INITIALIZER,HP_EMBEDDINGS_REGULARIZER,HP_EMBEDDINGS_CONSTRAINT,HP_ACTIVATION_D1, HP_ACTIVATION_D2, HP_OPTIMIZER,],
        metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')],)

In [20]:
def train_test_model(hparams):
    model = Sequential()
    model.add(Embedding(input_dim=num_words, 
                        output_dim=embedding_dim, 
                        mask_zero=True, 
                        embeddings_initializer=hparams[HP_EMBED_INITIALIZER], 
                        embeddings_regularizer=tf.keras.regularizers.l2(hparams[HP_EMBEDDINGS_REGULARIZER]), 
                        embeddings_constraint=hparams[HP_EMBEDDINGS_CONSTRAINT]))
    model.add(Bidirectional(GRU(8)))
    model.add(Dense(8, activation=hparams[HP_ACTIVATION_D1]))
    model.add(Dense(1, activation=hparams[HP_ACTIVATION_D2]))

    model.compile(loss='binary_crossentropy',optimizer=hparams[HP_OPTIMIZER],metrics='accuracy')
    
    history = model.fit(X_train_padded, y_train, epochs=10, validation_data=(X_val_padded, y_val))
    
    return np.max(history.history[METRIC_ACCURACY])

In [21]:
def run(run_dir, hparams):
    with tf.summary.create_file_writer(run_dir).as_default():
        hp.hparams(hparams)
        accuracy = train_test_model(hparams)
        tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)

In [22]:
session_num = 0
error_array = []

for i in range(10):
    try:
        hparams = {
        HP_EMBED_INITIALIZER: np.random.choice(HP_EMBED_INITIALIZER.domain.values),
        HP_EMBEDDINGS_REGULARIZER: np.random.uniform(HP_EMBEDDINGS_REGULARIZER.domain.min_value, HP_EMBEDDINGS_REGULARIZER.domain.max_value),
        HP_EMBEDDINGS_CONSTRAINT: np.random.choice(HP_EMBEDDINGS_CONSTRAINT.domain.values),
        HP_OPTIMIZER: np.random.choice(HP_OPTIMIZER.domain.values),
        HP_ACTIVATION_D1: np.random.choice(HP_ACTIVATION_D1.domain.values),
        HP_ACTIVATION_D2: np.random.choice(HP_ACTIVATION_D2.domain.values),
        }
    
        run_name = "run-%d" % session_num
        print('--- Starting trial: %s' % run_name)
        hparamsDict = {h.name: hparams[h] for h in hparams}
        print(hparamsDict)

        run('logs/hparam_tuning/' + run_name, hparams)
        session_num += 1
        
    except Exception as e:
        print(e)
        error_array.append(hparamsDict)
        continue

--- Starting trial: run-0
{'embeddings_initializer': 'Constant', 'l2 regularizer': 0.845184036581553, 'embeddings_constraint': 'radial_constraint', 'optimizer': 'adadelta', 'activation_d1': 'relu', 'activation_d2': 'softplus'}
Epoch 1/10
in user code:

    C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:806 train_function  *
        return step_function(self, iterator)
    C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:796 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\ProgramData\Anaconda3\lib\site-p

501/501 [==============================] - 25s 49ms/step - loss: 1.0252 - accuracy: 0.6082 - val_loss: 0.9507 - val_accuracy: 0.6353
--- Starting trial: run-1
{'embeddings_initializer': 'zeros', 'l2 regularizer': 0.7325045487515444, 'embeddings_constraint': 'min_max_norm', 'optimizer': 'nadam', 'activation_d1': 'softsign', 'activation_d2': 'softplus'}
Epoch 1/10
501/501 [==============================] - 28s 57ms/step - loss: 0.6885 - accuracy: 0.5580 - val_loss: 0.6838 - val_accuracy: 0.6243
Epoch 2/10
501/501 [==============================] - 18s 35ms/step - loss: 0.6674 - accuracy: 0.7088 - val_loss: 0.6521 - val_accuracy: 0.7394
Epoch 3/10
501/501 [==============================] - 19s 38ms/step - loss: 0.8485 - accuracy: 0.7009 - val_loss: 1.0131 - val_accuracy: 0.5474
Epoch 4/10
501/501 [==============================] - 21s 41ms/step - loss: 0.7328 - accuracy: 0.6135 - val_loss: 0.6726 - val_accuracy: 0.6453
Epoch 5/10
501/501 [==============================] - 19s 39ms/step - 

Epoch 1/10
in user code:

    C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:806 train_function  *
        return step_function(self, iterator)
    C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:796 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:789 run_step  **
  

Epoch 1/10
501/501 [==============================] - 30s 60ms/step - loss: 8.6300 - accuracy: 0.4341 - val_loss: 8.3497 - val_accuracy: 0.4525
Epoch 2/10
501/501 [==============================] - 23s 47ms/step - loss: 8.6300 - accuracy: 0.4341 - val_loss: 8.3497 - val_accuracy: 0.4525
Epoch 3/10
501/501 [==============================] - 22s 44ms/step - loss: 8.6300 - accuracy: 0.4341 - val_loss: 8.3497 - val_accuracy: 0.4525
Epoch 4/10
501/501 [==============================] - 22s 44ms/step - loss: 8.6300 - accuracy: 0.4341 - val_loss: 8.3497 - val_accuracy: 0.4525
Epoch 5/10
501/501 [==============================] - 22s 44ms/step - loss: 8.6300 - accuracy: 0.4341 - val_loss: 8.3497 - val_accuracy: 0.4525
Epoch 6/10
501/501 [==============================] - 20s 40ms/step - loss: 8.6300 - accuracy: 0.4341 - val_loss: 8.3497 - val_accuracy: 0.4525
Epoch 7/10
501/501 [==============================] - 19s 38ms/step - loss: 8.6300 - accuracy: 0.4341 - val_loss: 8.3497 - val_accuracy:

Epoch 1/10
501/501 [==============================] - 27s 54ms/step - loss: 26.7694 - accuracy: 0.4341 - val_loss: 14.4065 - val_accuracy: 0.4525
Epoch 2/10
501/501 [==============================] - 18s 36ms/step - loss: 14.6867 - accuracy: 0.4341 - val_loss: 14.4064 - val_accuracy: 0.4525
Epoch 3/10
501/501 [==============================] - 19s 37ms/step - loss: 14.6867 - accuracy: 0.4341 - val_loss: 14.4065 - val_accuracy: 0.4525
Epoch 4/10
501/501 [==============================] - 19s 37ms/step - loss: 14.6867 - accuracy: 0.4341 - val_loss: 14.4064 - val_accuracy: 0.4525
Epoch 5/10
501/501 [==============================] - 19s 38ms/step - loss: 14.6867 - accuracy: 0.4341 - val_loss: 14.4065 - val_accuracy: 0.4525 loss: 14.6419 - ac
Epoch 6/10
501/501 [==============================] - 16s 33ms/step - loss: 14.6867 - accuracy: 0.4341 - val_loss: 14.4064 - val_accuracy: 0.4525
Epoch 7/10
501/501 [==============================] - 18s 36ms/step - loss: 14.6867 - accuracy: 0.4341 - 

...

Epoch 1/10
501/501 [==============================] - 25s 50ms/step - loss: 6.7788 - accuracy: 0.5659 - val_loss: 6.9798 - val_accuracy: 0.5475
Epoch 2/10
501/501 [==============================] - 7216s 14s/step - loss: 6.6963 - accuracy: 0.5659 - val_loss: 6.9798 - val_accuracy: 0.5475
Epoch 3/10
501/501 [==============================] - 20s 39ms/step - loss: 6.6963 - accuracy: 0.5659 - val_loss: 6.9798 - val_accuracy: 0.5475
Epoch 4/10
501/501 [==============================] - 19s 38ms/step - loss: 6.6963 - accuracy: 0.5659 - val_loss: 6.9798 - val_accuracy: 0.5475
Epoch 5/10
501/501 [==============================] - 19s 37ms/step - loss: 6.6963 - accuracy: 0.5659 - val_loss: 6.9798 - val_accuracy: 0.5475
Epoch 6/10
501/501 [==============================] - 35498s 71s/step - loss: 6.6963 - accuracy: 0.5659 - val_loss: 6.9798 - val_accuracy: 0.5475
Epoch 7/10
501/501 [==============================] - 19s 38ms/step - loss: 6.6963 - accuracy: 0.5659 - val_loss: 6.9798 - val_accura

501/501 [==============================] - 19s 38ms/step - loss: 1.1454 - accuracy: 0.5944 - val_loss: 0.7780 - val_accuracy: 0.6020
Epoch 3/10
501/501 [==============================] - 19s 38ms/step - loss: 0.6577 - accuracy: 0.7066 - val_loss: 0.6067 - val_accuracy: 0.7334
Epoch 4/10
501/501 [==============================] - 18s 36ms/step - loss: 0.6054 - accuracy: 0.7429 - val_loss: 0.5949 - val_accuracy: 0.7424
Epoch 5/10
501/501 [==============================] - 18s 36ms/step - loss: 0.5874 - accuracy: 0.7495 - val_loss: 0.5901 - val_accuracy: 0.7396
Epoch 6/10
501/501 [==============================] - 17s 35ms/step - loss: 0.5814 - accuracy: 0.7532 - val_loss: 0.5835 - val_accuracy: 0.7574
Epoch 7/10
501/501 [==============================] - 18s 36ms/step - loss: 0.5734 - accuracy: 0.7610 - val_loss: 0.5834 - val_accuracy: 0.7505
Epoch 8/10
501/501 [==============================] - 21s 41ms/step - loss: 0.5727 - accuracy: 0.7689 - val_loss: 0.5860 - val_accuracy: 0.7727
Epo

--- Starting trial: run-41
{'embeddings_initializer': 'Identity', 'l2 regularizer': 0.6503135017486869, 'embeddings_constraint': 'max_norm', 'optimizer': 'adadelta', 'activation_d1': 'selu', 'activation_d2': 'tanh'}
Epoch 1/10
501/501 [==============================] - 38s 76ms/step - loss: 11.6128 - accuracy: 0.5659 - val_loss: 11.8658 - val_accuracy: 0.5475
Epoch 2/10
501/501 [==============================] - 24s 49ms/step - loss: 11.5957 - accuracy: 0.5659 - val_loss: 11.8469 - val_accuracy: 0.5475
Epoch 3/10
501/501 [==============================] - 23s 47ms/step - loss: 11.5757 - accuracy: 0.5659 - val_loss: 11.8244 - val_accuracy: 0.5475
Epoch 4/10
501/501 [==============================] - 19s 39ms/step - loss: 11.5524 - accuracy: 0.5659 - val_loss: 11.8039 - val_accuracy: 0.5475
Epoch 5/10
501/501 [==============================] - 22s 44ms/step - loss: 11.5302 - accuracy: 0.5659 - val_loss: 11.7852 - val_accuracy: 0.5475
Epoch 6/10
501/501 [==============================] - 

Epoch 1/10
501/501 [==============================] - 35s 69ms/step - loss: 0.7510 - accuracy: 0.4622 - val_loss: 0.7501 - val_accuracy: 0.4706
Epoch 2/10
501/501 [==============================] - 22s 43ms/step - loss: 0.7493 - accuracy: 0.4772 - val_loss: 0.7485 - val_accuracy: 0.4835
Epoch 3/10
428/501 [========================>.....] - ETA: 3s - loss: 0.7478 - accuracy: 0.49

KeyboardInterrupt: 

In [ ]:
!tensorboard --logdir logs/hparam_tuning

In [30]:
pd.Series(y_val.reshape(-1)).value_counts(normalize=True)

0    0.547548
1    0.452452
dtype: float64

There are a couple of issues with the models above. One is that certain hyperparameter settings are producing exceptions. The other is that some of the models appear to be producing only a single prediction. 54.75% of the values for y_val are 0's and 45.25% are 1's. We can see in the validation accuracy for some of the models above that they are stuck at either of these values for each epoch. 

If we can find out which hyperparameter settings are causing these, then we can remove these. This will speed up tuning and enable us to try a larger number of combinations.